In [2]:
import logging
import datetime
import json
import google.auth
import arrow
import json
import pandas as pd
import numpy as np
import time
from pandas.io.json import json_normalize
import google.auth
from google.cloud import bigquery
from google.auth.transport.requests import AuthorizedSession
import warnings
warnings.filterwarnings('ignore')

client = bigquery.Client(location="US", project="sb-bigdata-4985-da852265")

def get_api(url):
    import google.auth
    credentials, project = google.auth.default()
    from google.auth.transport.requests import AuthorizedSession
    authed_session = AuthorizedSession(credentials)
    response = authed_session.get(url)
    return response

def post_api(url, data):
    import google.auth
    credentials, project = google.auth.default()
    from google.auth.transport.requests import AuthorizedSession
    authed_session = AuthorizedSession(credentials)
    response = authed_session.post(url, data)
    return response

def get_createdby(createTime, jobId):
    createdBy="NA"
    import arrow
    import json
    createTime = arrow.get(createTime)
    logstart=createTime + datetime.timedelta(minutes = -2)
    logend=createTime + datetime.timedelta(minutes = 2)
    body = {"resourceNames": ["projects/sb-bigdata-4985-da852265"], "filter":'resource.type="audited_resource" \
    resource.labels.service="ml.googleapis.com" \
    resource.labels.method="google.cloud.ml.v1.JobService.CreateJob" \
    timestamp>="' + str(logstart) + '" \
    timestamp<="' + str(logend) + '"'
    }
    y = json.dumps(body)
    response=post_api('https://logging.googleapis.com/v2/entries:list', y)
    import json
    import pandas as pd
    data_df = pd.read_json(response.text)
    if 'entries' in data_df:
        for job in data_df['entries']:
            if job["protoPayload"]["request"]["job"]["jobId"] == jobId:
                createdBy=job["protoPayload"]["authenticationInfo"]["principalEmail"]
                break
    else:
        if response.status_code == 429:
            import time
            time.sleep(1)
            createdBy=get_createdby(createTime, jobId)
            
    return createdBy

def get_labels_df(createTime, jobId):
    labels="NA"
    import arrow
    import pandas as pd
    createTime = arrow.get(createTime)
    logstart=createTime + datetime.timedelta(minutes = -2)
    logend=createTime + datetime.timedelta(minutes = 2)
    body = {"resourceNames": ["projects/sb-bigdata-4985-da852265"], "filter":'resource.type="audited_resource" \
    resource.labels.service="ml.googleapis.com" \
    resource.labels.method="google.cloud.ml.v1.JobService.CreateJob" \
    timestamp>="' + str(logstart) + '" \
    timestamp<="' + str(logend) + '"'
    }
    y = json.dumps(body)
    response=post_api('https://logging.googleapis.com/v2/entries:list', y)
    data_df = pd.read_json(response.text)
    if 'entries' in data_df:
        for job in data_df['entries']:
            if job["protoPayload"]["request"]["job"]["jobId"] == jobId and "labels" in job["protoPayload"]["request"]["job"]:
                labels=job["protoPayload"]["request"]["job"]["labels"]
                break
    else:
        if response.status_code == 429:
            import time
            time.sleep(1)
            labels=get_labels_df(createTime, jobId)
            
    return labels

def get_trainingEndTime(createTime, job_Id):
    trainingEndTime="NA"
    page_token = ''
    createTime = arrow.get(createTime)
  
    filter_str = 'resource.labels.job_id="' + str(job_Id) + '" \
    jsonPayload.message="Module completed; cleaning up." OR "migration complete." OR "Execution halted"'
    body = {"resourceNames": ["projects/sb-bigdata-4985-da852265"],"pageToken" :str(page_token), "filter":filter_str }
    y = json.dumps(body)
    response=post_api('https://logging.googleapis.com/v2/entries:list', y) 
    if response.status_code == 200:     
        while True:
            data_df = pd.read_json(response.text, typ='series')
            if 'entries' in data_df and data_df['entries'][0]['resource']['labels']['job_id'] == job_Id:
                trainingEndTime = data_df['entries'][0]['timestamp']
                break    
            else:
                page_token = None
                page_token = data_df.get('nextPageToken')
                if page_token is None:
                    break
                else:
                    body = {"resourceNames": ["projects/sb-bigdata-4985-da852265"],"pageToken" :str(page_token), "filter":filter_str }
                    y = json.dumps(body)
                    response = post_api('https://logging.googleapis.com/v2/entries:list', y)

    return trainingEndTime
    
def get_jobs_df():
    job_list=[]
    regions=['us-west1', 'us-west2', 'us-central1', 'us-east1', 'us-east4']
    query = """
    SELECT MAX(createtime) as maxCreateTime
    FROM `sb-bigdata-4985-da852265.modelmanagement.job_metrics`;
    """
    query_job = client.query(query,   # Location must match that of the dataset(s) referenced in the query.
        location="US")  # API request - starts the query
    df = query_job.to_dataframe()
    response = get_api('https://ml.googleapis.com/v1/projects/sb-bigdata-4985-da852265/jobs')
    if response.status_code == 200:
        while True:
            import json
            import pandas as pd
            data_df = pd.read_json(response.text)
            for job in data_df['jobs']:
                if job['createTime'] > df['maxCreateTime'][0]:
                    job["trainingEndTime"]=get_trainingEndTime(job["createTime"], job["jobId"])
                    job["createdby"]=get_createdby(job["createTime"], job["jobId"])
                    job["labels"]=[get_labels_df(job["createTime"], job["jobId"])]
                    if job['state'] != "CANCELLED":
                        if any(x in job['trainingInput']['region'] for x in regions):
                            job["trainingCost"]=0.49*(job['trainingOutput']['consumedMLUnits'])
                        else:
                            job["trainingCost"]=0.54*(job['trainingOutput']['consumedMLUnits'])
                    else:
                        job["trainingCost"]="NA"
                    job_list.append(job)
                    print(job["jobId"])
                else:
                    None
            page_token = None
            page_token = data_df.get('nextPageToken')
            if page_token is None:
                break
            else:
                response = get_api('https://ml.googleapis.com/v1/projects/sb-bigdata-4985-da852265/jobs' + '?pageToken=' + str(page_token[0]))
    
    
    from pandas.io.json import json_normalize
    job_df=json_normalize(job_list)
    
    fields = pd.DataFrame(columns=['createTime', 'endTime', 'jobId', 'startTime', 'state', 'trainingInput.args', 'trainingInput.jobDir'
                                  , 'trainingInput.masterConfig.imageUri', 'createdby', 'trainingInput.masterType', 'trainingInput.packageUris'
                                  , 'trainingInput.pythonModule', 'trainingInput.pythonVersion', 'trainingInput.region', 'trainingInput.runtimeVersion'
                                  , 'trainingInput.scaleTier', 'trainingOutput.consumedMLUnits', 'labels', 'trainingInput.masterConfig.acceleratorConfig.count'
                                  , 'trainingInput.masterConfig.acceleratorConfig.type', 'trainingCost', 'trainingEndTime'])
    for col in fields.columns:
        if col not in job_df.columns:
            job_df[col] = np.nan
        else:
            None
    
    job_df_new = job_df[['createTime', 'endTime', 'jobId', 'startTime', 'state', 'trainingInput.args', 'trainingInput.jobDir'
                                  , 'trainingInput.masterConfig.imageUri', 'createdby', 'trainingInput.masterType', 'trainingInput.packageUris'
                                  , 'trainingInput.pythonModule', 'trainingInput.pythonVersion', 'trainingInput.region', 'trainingInput.runtimeVersion'
                                  , 'trainingInput.scaleTier', 'trainingOutput.consumedMLUnits', 'labels', 'trainingInput.masterConfig.acceleratorConfig.count'
                                  , 'trainingInput.masterConfig.acceleratorConfig.type', 'trainingCost', 'trainingEndTime']].copy()
    
    job_metrics_df = job_df_new.rename(columns={"createTime": "createtime", "endTime": "endtime", "jobId":"jobid", "startTime":"starttime", "state":"state"
                   , "trainingInput.args":"args", "trainingInput.jobDir":"jobdir", "trainingInput.masterConfig.imageUri":"imageuri", "createdby":"createdby"
                   , "trainingInput.masterType":"mastertype", "trainingInput.packageUris":"packageuris", "trainingInput.pythonModule":"pythonmodule"
                   , "trainingInput.pythonVersion":"pythonversion", "trainingInput.region":"region", "trainingInput.runtimeVersion":"runtimeversion"
                   , "trainingInput.scaleTier":"scaletier", "trainingOutput.consumedMLUnits":"consumedmlunits", "labels":"label"
                   , "trainingInput.masterConfig.acceleratorConfig.count":"acceleratorconfigcount"
                   , "trainingInput.masterConfig.acceleratorConfig.type":"acceleratorconfigtype", "trainingCost":"trainingcost"
                   , "trainingEndTime":"trainingendtime"})

    return job_metrics_df

def write_to_bqtable(bq_tablename, bq_schemaname, datadf):
    import google.auth
    from google.cloud import bigquery
    credentials, project = google.auth.default()
    client = bigquery.Client(project)
    job_config = bigquery.LoadJobConfig()
    job_config.schema = bq_schemaname
    
    job = client.load_table_from_dataframe(datadf, bq_tablename, job_config=job_config)
    # Wait for the load job to complete.
    job.result()

def get_data(request):
    jobs = []
    jobs = get_jobs_df()
    print("AI platform jobs count : ",str(len(jobs)))
    jobs.insert(0, 'date', str(datetime.datetime.utcnow()))
    from google.cloud import bigquery
    varbqschema=[
        bigquery.SchemaField(name="date", field_type="STRING"),
        bigquery.SchemaField(name="createtime", field_type="STRING"),
        bigquery.SchemaField(name="createdby", field_type="STRING"),
        bigquery.SchemaField(name="jobid", field_type="STRING"),
        bigquery.SchemaField(name="starttime", field_type="STRING"),
        bigquery.SchemaField(name="endtime", field_type="STRING"),
        bigquery.SchemaField(name="trainingendtime", field_type="STRING"),
        bigquery.SchemaField(name="state", field_type="STRING"),
        bigquery.SchemaField(name="args", field_type="STRING"),
        bigquery.SchemaField(name="jobdir", field_type="STRING"),
        bigquery.SchemaField(name="imageuri", field_type="STRING"),
        bigquery.SchemaField(name="mastertype", field_type="STRING"),
        bigquery.SchemaField(name="packageuris", field_type="STRING"),
        bigquery.SchemaField(name="pythonmodule", field_type="STRING"),
        bigquery.SchemaField(name="pythonversion", field_type="STRING"),
        bigquery.SchemaField(name="region", field_type="STRING"),
        bigquery.SchemaField(name="runtimeversion", field_type="STRING"),
        bigquery.SchemaField(name="scaletier", field_type="STRING"),
        bigquery.SchemaField(name="consumedmlunits", field_type="STRING"),
        bigquery.SchemaField(name="label", field_type="STRING"),
        bigquery.SchemaField(name="acceleratorconfigcount", field_type="STRING"),
        bigquery.SchemaField(name="acceleratorconfigtype", field_type="STRING"),
        bigquery.SchemaField(name="trainingcost", field_type="STRING")
    ]
    write_to_bqtable('modelmanagement.job_metrics', varbqschema, jobs.astype(str))
    return "Done!"

In [3]:
print(datetime.datetime.now())
df = get_data(None)
print(datetime.datetime.now())

2020-01-27 10:39:56.786374
TPU_fashion_MNIST_20200124_144703
TPU_fashion_MNIST_20200123_124443
TPU_fashion_MNIST_20200123_121525
sentiment__TPU_resolver_strategy_114_20200122_182453
sentiment_py3_basic_TPU_20200122_181656
sentiment_py3_basic_TPU_20200122_171253
sentiment_py3_basic_TPU_20200122_170634
sentiment_py3_basic_TPU_20200122_170131
sentiment_py3_basic_TPU_20200122_165204
sentiment_py3_basic_TPU_20200122_164413
sentiment_py3_basic_TPU_20200122_162936
sentiment_py3_basic_TPU_20200122_162453
sentiment_py3_basic_TPU_20200122_161452
sentiment_py3_basic_TPU_20200122_160857
sentiment_py3_basic_TPU_20200122_160433
AI platform jobs count :  15
2020-01-27 10:43:09.676455
